# PVE-analysis on research funding
*Collective Compass Project, DJA, July 2022*

## Loading the necessary Python modules.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import itertools
plt.rcParams.update({'font.size': 14}) # must set in top
plt.rcParams.update({'figure.figsize': (14, 4)}) # must set in top

---
## Reading the data from disk

In [ ]:
# 550 M is the total budget
df1 = pd.read_csv('Property values 1.csv', sep=';')

In [ ]:
# 400 M is the total budget
df2 = pd.read_csv('Property values 2.csv', sep=';')

In [ ]:
# 900 M is the total budget
df3 = pd.read_csv('Property values 3.csv', sep=';')

In [ ]:
df = pd.read_stata('DJA_forDJA.dta')

---
## Correct the data where needed

In [ ]:
df['inst'] = df['inst'].replace('Delft University of Technology', 'Technische Universiteit Delft')

---
# Plot basic participation stats per stratification of choice

In [ ]:
plt.figure()
plt.subplot(121)
df['inst'].value_counts().plot.bar()
plt.grid()
plt.subplot(122)
df['versie'].value_counts().plot.bar();

In [ ]:
df_vals = df[['inst', 'versie']].value_counts()
df_vals.unstack('versie').plot.bar(stacked=True);

In [ ]:
df_vals = df[['pos5', 'gender']].value_counts()
df_vals.unstack('gender').plot.bar(stacked=True);

---
## Postprocess the kt values to fraction of the total budget.

In [ ]:
# First, we read the actual maximum budget values of which the kt values are a fraction from the table.
df1_abs_vals = df1.iloc[0:11, 2].values
df2_abs_vals = df2.iloc[0:11, 2].values
df3_abs_vals = df3.iloc[0:11, 2].values

In [ ]:
# Second, we multiply the kt values with the values above and normalize with the total
# to make each kt a fraction of the total money spend.
df_kt = df[['versie', 'ktversie', 'kt1', 'kt2', 'kt3', 'kt4', 'kt5', 'kt6', 'kt7', 'kt8', 'kt9', 'kt10', 'kt11']].copy()
df_kt['success_chance'] = 'zero'
df_kt['invested_time'] = 'lots'

print(df_kt['invested_time'])

for ikt in range(11):
    key = 'kt{}'.format(ikt+1)
    mask = df_kt['versie'] == '1'
    df_kt.loc[mask, key] = df_kt.loc[mask, key] * df1_abs_vals[ikt] / 550.
       
for ikt in range(11):
    key = 'kt{}'.format(ikt+1)
    mask = df_kt['versie'] == '2'
    df_kt.loc[mask, key] = df_kt.loc[mask, key] * df2_abs_vals[ikt] / 400.
    
for ikt in range(11):
    key = 'kt{}'.format(ikt+1)
    mask = df_kt['versie'] == '3'
    df_kt.loc[mask, key] = df_kt.loc[mask, key] * df3_abs_vals[ikt] / 900.

In [ ]:
# Create a table with the fraction spent per kt item per group.
df_kt_per_version = df_kt.groupby('versie')[['kt1', 'kt2', 'kt3', 'kt4', 'kt5', 'kt6', 'kt7', 'kt8', 'kt9', 'kt10', 'kt11', 'success_chance', 'invested_time']]
df_kt_per_version.mean()

In [ ]:
# Print the fraction of the total money spent per experiment version.
df_kt_per_version.mean().sum(axis=1)

In [ ]:
# Plot a histogram with a bar per version.
df_kt_per_version.mean().transpose().plot.bar()
plt.grid()

In [ ]:
# Plot a histogram per version.
df_kt_per_version.mean().plot.bar()
plt.grid()